In [1]:
from simCAS import *

In [2]:
import numpy as np
from scipy.stats import multivariate_normal
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import pandas as pd
from sklearn.neighbors import KernelDensity
import os
from scipy import sparse
import scipy.io as sio
import scanpy as sc
from Bio import Phylo
from io import StringIO
import logging
from scipy.optimize import fsolve
import random
import threading
import scipy.stats as stats
from sklearn.mixture import GaussianMixture as GMM
from scipy.stats import logser

from scipy.special import rel_entr
from statsmodels.discrete.count_model import (ZeroInflatedNegativeBinomialP, ZeroInflatedPoisson,
                                              ZeroInflatedGeneralizedPoisson)
import statsmodels.api as sm
from scipy.stats import nbinom
from scipy.special import expit

In [3]:
#读取参数
prefix_='Buenrostro_2018'
# prefix_='Forebrain'
# prefix_='MCA/Cerebellum'
resultdir='/data1/lichen/code/second/scATAC_integration/data/scATACdata_total/process/{0}/'.format(prefix_)
peak_mean=pd.read_csv(resultdir+'peak_mean_log.csv',index_col=0)
lib_size=pd.read_csv(resultdir+'library_size_log.csv',index_col=0)
nozero=pd.read_csv(resultdir+'nozero_log.csv',index_col=0)

peak_mean=np.array(peak_mean['peak mean'])
lib_size=np.array(lib_size['library size'])
nozero=np.array(nozero['nozero'])

In [14]:
# 设置参数，后续所有函数中的参数都在下面给出定义

n_peak         =len(peak_mean) # peak数目
n_cell_total   =1500 #总共的细胞数目
rand_seed      =2022 #随机种子
zero_prob      =0.5  #对于peak_effect的置零个数
zero_set       ='all'#'by_row'指的是对于每一个peak的effect vector进行置零；'all'指的是随机在所有的index中选择进行置零
effect_mean    =0 #生成effect vector的均值
effect_sd      =1 #生成effect vector的方差

min_popsize    =300 #离散模式下设定的细胞群的最小数目
min_pop        ='A' #离散模式下设定最小细胞群的名称，注意需要与下面的tree_text一致
tree_text      =["((A:1,B:1):1,(C:1,D:1):1);", #注：前三个用来仿真离散模式，只标叶子结点名称就行；后两个为连续模式的仿真树，与标准newick形式略有不同
                 "(A:1,(C:1,D:1):1);",
                '(((A:1, B:1):1,(C:0.2, D:0.2):1):1, E:3);',
                '((((A:1, B:1)C:1,(D:0.2, E:0.2)F:1)G:1, H:3)S)',
                "(((A:1,B:1)C:1,(D:1,E:1)F:1)S)"]
pops_name      =[['A','B','C','D'],
                ['A','C','D'],
                ['A','B','C','D','E']]  # 输入不同节点的名字，离散模式只需要输入叶子节点的名称就行，注意这里需要与tree_text的前三个顺序保持一致
pops_size      =None # 设置不同cluster的细胞数目，None则直接取平均


embed_mean_same=1 # 对embedding非差异特征采样的均值
embed_sd_same  =0.5 # 对embedding的非差异特征采样的方差
embed_mean_diff=1 # 对embedding差异特征采样的均值
embed_sd_diff  =0.5 # 对embedding的差异特征采样的方差

len_cell_embed =12   #仿真细胞的低维特征的特征个数
n_embed_diff   =10 # 使得cell embedding不同的特征维度数目
n_embed_same   =len_cell_embed-n_embed_diff

simu_type      ='cell_type' # continuous/discrete/single/cell_type
correct_iter   =2 # 使用参数进行修正的迭代次数
activation     ='sigmod' #对参数矩阵矫正的方式，在连续和离散的条件下使用'exp'，在仿真celltype的时候应该使用'sigmod'

two_embeds     =True  # true表明peak mean和library size通过两个不同的矩阵排序对应得到；False 表明通过一个矩阵的值排序对应得到

adata_dir      =resultdir+'adata_forsimulation.h5ad' # 为了进行cell_type simulation
lib_simu       ='estimate' # 在仿真cell_type时用的参数，’real‘表示直接使用真实的library_size参数，‘estimate’表示从估计的分布中采样
distribution   ='Poisson' # 数据的分布，如果二值化就是’Bernoulli‘，count就是‘Poisson’

bw_pm          =1e-4 #分别为对peak mean、library_size、nozero的核密度估计的窗宽；注：bw_pm若取的过大可能会导致采样的peak mean小于0而报错
bw_lib         =0.05
bw_nozero      =0.05

real_param     =False # 是否使用真实的参数，True则为直接使用真实参数，False

log            =None

fix_seed(rand_seed)

k_dict,pi_dict={},{}

In [15]:
# 生成effect和embedding
print("**********start generate effect vector...**********")
peak_effect,lib_size_effect=Get_Effect(n_peak,n_cell_total,
                len_cell_embed,rand_seed,zero_prob,zero_set,effect_mean,effect_sd)
print("**********generate effect finished!**********")



print("**********start generate cell embedding...**********")
print("simulation type is {0}".format(simu_type))
if simu_type=='discrete':
    # 重复两次获得两个矩阵，后续使用参数two_embeds决定是用两个矩阵还是用一个
    embeds_peak,meta=Get_Discrete_Embedding(pops_name[2],min_popsize,tree_text[2],
                 n_cell_total,pops_size,
                 embed_mean_same,embed_sd_same,
                  embed_mean_diff,embed_sd_diff,
                 n_embed_diff,n_embed_same,rand_seed)
    embeds_lib,meta=Get_Discrete_Embedding(pops_name[2],min_popsize,tree_text[2],
                 n_cell_total,pops_size,
                 embed_mean_same,embed_sd_same,
                  embed_mean_diff,embed_sd_diff,
                 n_embed_diff,n_embed_same,rand_seed+1)
    embeds_peak,embeds_lib=embeds_peak.values,embeds_lib.values
    print("**********generate cell embedding finished**********")
    # 获得count
    atac_counts=Get_Tree_Counts(peak_mean,lib_size,nozero,n_peak,n_cell_total,rand_seed,
                    embeds_peak,embeds_lib,correct_iter,distribution,activation,bw_pm,bw_lib,bw_nozero,
                                real_param)
    print("**********generate counts finshed!**********")
    
elif simu_type=='continuous':
    embeds_param={}
    embeds_peak,meta=Get_Continuous_Embedding(tree_text[4],n_cell_total,
                 embed_mean_same,embed_sd_same,
                  embed_mean_diff,embed_sd_diff,
                 n_embed_diff,n_embed_same,rand_seed)
    embeds_lib,meta=Get_Continuous_Embedding(tree_text[4],n_cell_total,
                 embed_mean_same,embed_sd_same,
                  embed_mean_diff,embed_sd_diff,
                 n_embed_diff,n_embed_same,rand_seed+1)
    embeds_peak,embeds_lib=embeds_peak.values,embeds_lib.values
    print("**********generate cell embedding finished**********")
    
    print("**********start generate counts...**********")
    atac_counts=Get_Tree_Counts(peak_mean,lib_size,nozero,n_peak,n_cell_total,rand_seed,
                    embeds_peak,embeds_lib,correct_iter,distribution,activation,bw_pm,bw_lib,bw_nozero,
                               real_param)
    print("**********generate counts finshed!**********")
    
elif simu_type=='single':
    embeds_param={}
    embeds_peak,meta=Get_Single_Embedding(n_cell_total,embed_mean_same,embed_sd_same,
                 n_embed_diff,n_embed_same)
    embeds_lib,meta=Get_Single_Embedding(n_cell_total,embed_mean_same,embed_sd_same,
                 n_embed_diff,n_embed_same)
    embeds_peak,embeds_lib=embeds_peak.values,embeds_lib.values
    print("**********generate cell embedding finished!**********")
    
    
    print("**********start generate counts...**********")
    atac_counts=Get_Tree_Counts(peak_mean,lib_size,nozero,n_peak,n_cell_total,rand_seed,
                    embeds_peak,embeds_lib,correct_iter,distribution,activation,bw_pm,bw_lib,bw_nozero)
    print("**********generate counts finshed!**********")
    
elif simu_type=='cell_type':
    adata=sc.read_h5ad(adata_dir)
    counts_list,celltype_list,embed_peak_list,embed_lib_list=[],[],[],[]
    lambdas_list,simu_param_nozero_list,simu_param_lib_list,simu_param_pm_list=[],[],[],[]#新加的list用来重新对lambdas进行spasity的修正
    celltypes=np.unique(adata.obs.celltype)
    for i in range(len(celltypes)):
    # 可以分为直接从真实数据中进行采样或是从核密度估计中采样特定细胞数目，先做直接从真实数据中采样的结果
        # print(celltypes[i])
        print("simulating cell type: {}...".format(celltypes[i]))
        adata_part=adata[adata.obs.celltype==celltypes[i],:]

        # 对每个celltype单独进行仿真
        counts,embed_peak,embed_lib,lambdas,simu_param_nozero,simu_param_lib,simu_param_pm=Get_Celltype_Counts(adata_part,two_embeds,
                                            embed_mean_same,embed_sd_same,len_cell_embed,effect_mean,effect_sd,
                     n_embed_diff,n_embed_same,correct_iter,lib_simu=lib_simu,n_cell_total=None,
                                        distribution=distribution,activation=activation,
                    bw_pm=bw_pm,bw_lib=bw_lib,bw_nozero=bw_nozero,rand_seed=rand_seed,zero_prob=zero_prob,zero_set=zero_set) # peak*cell

        counts_list.append(counts)
        embed_peak_list.append(embed_peak)
        embed_lib_list.append(embed_lib)
        celltype_list.append([celltypes[i]]*counts.shape[1])
        lambdas_list.append(lambdas)
        simu_param_nozero_list.append(simu_param_nozero)
        simu_param_lib_list.append(simu_param_lib)
        simu_param_pm_list.append(simu_param_pm)
        
    if distribution=='Poisson':
        # atac_counts=np.hstack(counts_list)
        meta=np.hstack(celltype_list)
        embeds_peak=np.hstack(embed_peak_list)
        embeds_lib=np.hstack(embed_lib_list)
        #对整体lambdas进行sparsity修正
        lambdas=np.hstack(lambdas_list)
        simu_param_nozero=np.hstack(simu_param_nozero_list)
        simu_param_lib=np.hstack(simu_param_lib_list)
        simu_param_pm=peak_mean

        lambdas_sum=np.sum(lambdas,axis=0)
        
        
        n_cell_total=len(simu_param_lib)
        print("**********start ZIP correction...**********")
        k_list,pi_list=[],[]
        # 求解每个cell中lambda扩大的倍数和置零的比例
        for i in range(n_cell_total):
            iter_=i
            # print(i)
            def solve_function(unsolved_value):
                k,pi=unsolved_value[0],unsolved_value[1]
                return [
                    k*(1-pi)-simu_param_lib[iter_]/(lambdas_sum[iter_]),
                    n_peak*pi+(1-pi)*np.sum(np.exp(-lambdas[:,iter_]*k))-(n_peak-simu_param_nozero[iter_])
                ]

            solved=fsolve(solve_function,[3,0.5],maxfev=2000)
            k,pi=solved[0],solved[1]
            simu1=k*(1-pi)*(lambdas_sum[iter_])
            real1=simu_param_lib[iter_]
            if abs(simu1-real1)/real1>0.1:
                print('=================================')
                print(i)
                print(simu1,real1)
                solved=fsolve(solve_function,[20,0.5],maxfev=2000)
            k,pi=solved[0],solved[1]
            simu1=k*(1-pi)*(lambdas_sum[iter_])
            real1=simu_param_lib[iter_]
            if abs(simu1-real1)/real1>0.1:
                print(i)
                print(simu1,real1)
                print('=================================')
            k_list.append(solved[0])
            pi_list.append(solved[1])
        # 对每个cell的lambda置零并扩大相应倍数
        for i in range(n_cell_total):
            if k_list[i]==3 or k_list[i]==20 or pi_list[i]<0 or k_list[i]<0:
                continue
            a=lambdas[:,i]*k_list[i]
            # b=atac_counts[:,i]
            a[np.random.choice(n_peak,replace=False,size=int(pi_list[i]*n_peak))]=0
            lambdas[:,i]=a
        print("**********ZIP correction finished!**********")
        
#         n_cell_total=len(simu_param_lib)
#         print("**********start ZIP correction...**********")
#         batch_size = 1000 # 并行数目，全局字典
#         global k_dict,pi_dict
#         for i in range(0,n_cell_total,batch_size):
#             if i+batch_size<=n_cell_total:
#                 my_thread = [zip_correction_thread(j,simu_param_lib[j],lambdas[:,j],lambdas_sum[j],simu_param_nozero[j],n_peak) for j in range(i, i+batch_size)]
#             else:
#                 my_thread = [zip_correction_thread(j,simu_param_lib[j],lambdas[:,j],lambdas_sum[j],simu_param_nozero[j],n_peak) for j in range(i, n_cell_total)]
#             for thread_ in my_thread:
#                 thread_.start()
#             for thread_ in my_thread:
#                 thread_.join()
#         # 对每个cell的lambda置零并扩大相应倍数
#         for i in range(n_cell_total):
#             if k_dict[i]==3 or k_dict[i]==20 or pi_dict[i]<0 or k_dict[i]<0:
#                 continue
#             a=lambdas[:,i]*k_dict[i]
#             # b=atac_counts[:,i]
#             a[np.random.choice(n_peak,replace=False,size=int(pi_dict[i]*n_peak))]=0
#             lambdas[:,i]=a
            
#         print("**********ZIP correction finished!**********")

        # # spasity矫正完之后再来一轮peak mean和library size的矫正，保证都符合实际
        # lambdas_copy=lambdas.copy()
        # lambdas_copy=lambdas_copy/(np.sum(lambdas_copy,axis=1).reshape(-1,1)+1e-8)*(simu_param_pm.reshape(-1,1))*lambdas_copy.shape[1]
        # lambdas_copy=lambdas_copy/(np.sum(lambdas_copy,axis=0).reshape(1,-1)+1e-8)*(simu_param_lib.reshape(1,-1))

        atac_counts=np.random.poisson(lambdas, lambdas.shape)
        
    elif distribution=='Bernoulli':
        atac_counts=np.hstack(counts_list)
        meta=np.hstack(celltype_list)
        embeds_peak=np.hstack(embed_peak_list)
        embeds_lib=np.hstack(embed_lib_list)
        
    else:
        raise ValueError('wrong distribution input!')
    
    print("**********generate counts finshed!**********")

else:
    raise ValueError('wrong simulation type!')

**********start generate effect vector...**********
**********generate effect finished!**********
**********start generate cell embedding...**********
simulation type is cell_type
simulating cell type: CLP...
the estimation method is  zero_NB
Optimization terminated successfully.
         Current function value: 3.193879
         Iterations: 9
         Function evaluations: 10
         Gradient evaluations: 10
correct_iter 1
correct_iter 2
simulating cell type: CMP...
the estimation method is  zero_NB
Optimization terminated successfully.
         Current function value: 4.615368
         Iterations: 12
         Function evaluations: 13
         Gradient evaluations: 13
correct_iter 1
correct_iter 2
simulating cell type: GMP...
the estimation method is  zero_NB
Optimization terminated successfully.
         Current function value: 4.656292
         Iterations: 12
         Function evaluations: 13
         Gradient evaluations: 13
correct_iter 1
correct_iter 2
simulating cell type: HSC.

/data1/lichen/anaconda3/envs/pytorch2/lib/python3.8/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


10
1463.8499582193344 2611.259113901049
29
1010.0406231258172 1848.7352287845374


/data1/lichen/anaconda3/envs/pytorch2/lib/python3.8/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 2000.
  warnings.warn(msg, RuntimeWarning)


75
1017.4655821535761 1943.8089746153423
82
2082.164644758162 3969.2000921090557
84
3873.8058421518426 7155.309382055134
88
962.7107282736833 1757.9315721294358
88
827.3204434663775 1757.9315721294358
89
1848.9068703746252 3558.727370949939
90
2188.1322751220887 4254.899671147131
93
2892.7583968832514 5853.66277450764
95
4486.8168395175 8368.612237912112
98
1282.5182660745872 2413.299300072821
102
2062.6446496122862 4272.062688254692
111
2044.8498908842118 4189.954924036229
112
1389.8629786775084 2607.6691329183705
118
2163.0353015808423 4112.214851792217
121
2030.6877880733703 4078.9612451111875
123
1228.8079856328839 2299.7116968277346
125
2322.440035041767 4677.5317707805225
127
2579.0162099142854 5415.0589296651015
132
2102.8215654906317 3992.7062036832863
133
3602.00195897575 6502.306216719294
137
1708.0495130377133 3247.619623443845
138
3985.894128056913 7990.565474243419
140
1478.8057074432495 2770.9265810422335
141
3545.1811329767056 7083.290685316798
144
2072.812662082847 4045

In [16]:
np.max(atac_counts)

211

In [9]:
a.shape,pi_list[i]*n_peak,pi_list[i],k_dict[i]

((169221,), 191976.3752400068, 1.1344713436276042, -3.591242197603106)

In [17]:
import test

In [18]:
test.change()

In [19]:
test.k_dict

{'1': 1}